# **Build RAG app using Replicate API, LlamaIndex, and Pinecone**

### **1. Install dependencies**

In [ ]:
!pip install llama-index llama-index-vector-stores-pinecone llama-index-llms-replicate pinecone python-dotenv

### **2. Importing Libraries**

In [2]:
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext, Settings
from llama_index.vector_stores.pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.replicate import Replicate
from dotenv import load_dotenv

load_dotenv()

u:\testLlamaIndex\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

### **3. Set environment variables and api keys**

In [ ]:
Replicate_API_KEY = os.environ.get("REPLICATE_API_TOKEN")
Pinecone_API_KEY = os.environ.get("PINECONE_API")

os.environ["OPENAI_API_KEY"] = (
    "your_openai_api_key"
)

### **4. Create Pinecone index**

In [ ]:
# Initialize a client
pc = Pinecone(api_key=Pinecone_API_KEY)

# Create a serverless index
index_name = "your_index_name"
dimension = 1536

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
    ) 


pc.describe_index(index_name)

### **5. Connect to the index**

In [ ]:
Pinecone_Host = os.environ.get("INDEX_HOST")
pinecone_index = pc.Index(host=Pinecone_Host)

pc.list_indexes()

### **6. LLM configuration for embedding and generation**

In [6]:
from tkinter import SE


embed_model = OpenAIEmbedding(model="text-embedding-3-large", dimensions=1536)
llm = Replicate(model="meta/meta-llama-3-8b-instruct", temperature=0.7)

Settings.embed_model = embed_model
Settings.llm = llm

### **7. Load documents, build the PineconeVectorStore and VectorStoreIndex**

In [15]:
# load documents
documents = SimpleDirectoryReader("data").load_data()

# Create a PineconeVectorStore using the specified pinecone_index
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)

# Create a StorageContext using the created PineconeVectorStore
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Use the chunks of documents and the storage_context to create the index
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

Upserted vectors: 100%|██████████| 68/68 [00:05<00:00, 11.51it/s]


## *Alternatively, if you want to load your "Existing" Pinecone Vector Store*

In [7]:
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)

index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

### **8. Query Index**

In [8]:
chat_engine = index.as_chat_engine(chat_mode="context", verbose=True)

In [9]:
from IPython.display import Markdown, display

response = chat_engine.chat("How do I prepare myself for interview. What are the mendatory questions that I should prepare for?")
display(Markdown(f"<b>{response}</b>"))

<b>

Congratulations on taking the first step towards preparing for your interview! Based on the guidelines provided, here are some key takeaways to help you prepare:

1. Rehearse your answers: Review the general guidelines and the specific question and answer format provided. Practice answering the questions using the suggested framework.
2. Research the employer's needs: Before the interview, research the company and the position to understand what the employer is looking for. Ask questions to uncover their needs, wants, and goals.
3. Prepare key questions: Prepare a list of questions to ask the interviewer, such as "What are the most important priorities of this position?" or "Can you tell me more about the company culture?"
4. Focus on matching your qualifications to the employer's needs: Emphasize how your skills, experience, and achievements match the employer's needs.
5. Practice being optimistic and positive: As a daily exercise, practice being more optimistic and putting a positive spin on events and situations.
6. Be honest and authentic: Remember to be honest and authentic in your answers, and avoid lying or exaggerating.

As for the mandatory questions you should prepare for, the article mentions that about 80% of all interviews begin with the question "Tell me about yourself." Here's a suggested answer format:

* Start with the present and highlight your qualifications for the position.
* Show how your skills, experience, and achievements match the employer's needs.
* Use specific examples to illustrate your responsibilities and achievements.

Additionally, be prepared to answer behavioral questions, such as "Can you give an example of a time when...?" or "How did you handle a difficult situation in the past?"

Remember to stay calm, confident, and authentic during the interview, and to focus on showcasing your skills and qualifications. Good luck!</b>

### **9. Context Retrieval/Similarity Search**

In [10]:
retriever = index.as_retriever(similarity_top_k=3)

In [11]:
nodes = retriever.retrieve("What is Golfergpt?")

for i, node in enumerate(nodes):
    print(f"Node {i+1}: Score: {node.score}\nText: {node.text}\n")

Node 1: Score: 0.45851323
Text: GolferGPT Assistance: 
Our GolferGPT is here to assist you in many ways! If you ever face issues like: 
1. Changing profile information? – Need help updating your name, photo, or other details? 
GolferGPT has your back. 
2. Creating or interacting with a post? – Whether it’s writing, editing, liking, or commenting, 
GolferGPT will guide you through it. 
3. Creating or following a tournament? – Confused about how to host or join a tournament? 
Just ask GolferGPT! 
4. Inviting loved ones? – Want to bring friends or family into the app? GolferGPT can walk 
you through the invitation process. 
5. Managing friendships? – From sending friend requests to unfriending, GolferGPT makes 
it easy. 
6. Finding a good spot to golf? – Looking for a nearby or popular golf location? GolferGPT 
can help with suggestions and info. 
No worries — GolferGPT is always ready to assist you with anything you need in the app! 
 
NOTIFICATIONS 
Overview: 
The Notifications feature 

In [12]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine(retriever=retriever)
response = query_engine.query("How can I create a new post?")
print(response)



To create a new post, follow these steps:

1. On the Home Screen, tap the "Got something to say? +" field next to your profile picture.
2. This opens the Post Creation Screen where you can customize and publish your post.

You can then customize your post by choosing the post visibility options, adding text, photos, videos, tagging people, and more.


## **> You can inspect the retrieved node**

In [13]:
from llama_index.core.response.pprint_utils import pprint_source_node

pprint_source_node(nodes[0])

Node ID: 7d4f9e8d-2741-4fe4-bf15-5ff829ca0111
Similarity: 0.45851323
Text: GolferGPT Assistance:  Our GolferGPT is here to assist you in
many ways! If you ever face issues like:  1. Changing profile
information? – Need help updating your name, photo, or other details?
GolferGPT has your back.  2. Creating or interacting with a post? –
Whether it’s writing, editing, liking, or commenting,  GolferGPT will
guide you thro...


In [14]:
nodes[0].node.metadata

{'page_label': '11',
 'file_name': 'doc1.pdf',
 'file_path': 'u:\\testLlamaIndex\\data\\doc1.pdf',
 'file_type': 'application/pdf',
 'file_size': 4201029,
 'creation_date': '2025-07-21',
 'last_modified_date': '2025-05-05'}